In [1]:
from maud.converters import DoclingConverterAdapter
from pathlib import Path

In [3]:
input_path = Path('../data/input/92ab18cd-55d7-5c30-a347-2369dd751f74.pdf')
output_dir = Path('../data/output')

converter = DoclingConverterAdapter(input_path, output_dir)

converter.convert()

2025-02-04 17:08:40,756 - DoclingConverterAdapter - INFO - Converting document


DoclingDocument(schema_name='DoclingDocument', version='1.0.0', name='92ab18cd-55d7-5c30-a347-2369dd751f74', origin=DocumentOrigin(mimetype='application/pdf', binary_hash=12584345842624200960, filename='92ab18cd-55d7-5c30-a347-2369dd751f74.pdf', uri=None), furniture=GroupItem(self_ref='#/furniture', parent=None, children=[], name='_root_', label=<GroupLabel.UNSPECIFIED: 'unspecified'>), body=GroupItem(self_ref='#/body', parent=None, children=[RefItem(cref='#/pictures/0'), RefItem(cref='#/texts/0'), RefItem(cref='#/groups/0'), RefItem(cref='#/texts/1'), RefItem(cref='#/texts/2'), RefItem(cref='#/pictures/1'), RefItem(cref='#/groups/1'), RefItem(cref='#/pictures/2'), RefItem(cref='#/texts/4'), RefItem(cref='#/texts/5'), RefItem(cref='#/texts/6'), RefItem(cref='#/texts/7'), RefItem(cref='#/texts/8'), RefItem(cref='#/texts/9'), RefItem(cref='#/pictures/3'), RefItem(cref='#/pictures/4'), RefItem(cref='#/pictures/5'), RefItem(cref='#/texts/10'), RefItem(cref='#/texts/11'), RefItem(cref='#/te

In [5]:
converter.document

DoclingDocument(schema_name='DoclingDocument', version='1.0.0', name='92ab18cd-55d7-5c30-a347-2369dd751f74', origin=DocumentOrigin(mimetype='application/pdf', binary_hash=12584345842624200960, filename='92ab18cd-55d7-5c30-a347-2369dd751f74.pdf', uri=None), furniture=GroupItem(self_ref='#/furniture', parent=None, children=[], name='_root_', label=<GroupLabel.UNSPECIFIED: 'unspecified'>), body=GroupItem(self_ref='#/body', parent=None, children=[RefItem(cref='#/pictures/0'), RefItem(cref='#/texts/0'), RefItem(cref='#/groups/0'), RefItem(cref='#/texts/1'), RefItem(cref='#/texts/2'), RefItem(cref='#/pictures/1'), RefItem(cref='#/groups/1'), RefItem(cref='#/pictures/2'), RefItem(cref='#/texts/4'), RefItem(cref='#/texts/5'), RefItem(cref='#/texts/6'), RefItem(cref='#/texts/7'), RefItem(cref='#/texts/8'), RefItem(cref='#/texts/9'), RefItem(cref='#/pictures/3'), RefItem(cref='#/pictures/4'), RefItem(cref='#/pictures/5'), RefItem(cref='#/texts/10'), RefItem(cref='#/texts/11'), RefItem(cref='#/te